In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time
import pickle
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import mean_squared_error as mse
from sklearn.ensemble import VotingRegressor,RandomForestRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression
from sklearn import model_selection as ms
from scipy.stats import pearsonr
from sklearn.metrics import make_scorer
from sklearn import metrics
from sklearn.feature_selection import VarianceThreshold

rand_st=4

In [2]:
output=pd.read_excel(r'C:\Users\Asus\PredictingRatings\models\class_results.xlsx')
errors=pd.read_excel(r'C:\Users\Asus\PredictingRatings\models\class_errors.xlsx')

In [3]:
elos_1=pd.read_csv(r'C:\Users\Asus\PredictingRatings\data\elos1.csv')
elos_2=pd.read_csv(r'C:\Users\Asus\PredictingRatings\data\elos2.csv')
elos_3=pd.read_csv(r'C:\Users\Asus\PredictingRatings\data\elos3.csv')
elos_4=pd.read_csv(r'C:\Users\Asus\PredictingRatings\data\elos4.csv')
features_1=pd.read_csv(r'C:\Users\Asus\PredictingRatings\data\features1.csv')
features_3=pd.read_csv(r'C:\Users\Asus\PredictingRatings\data\features3.csv')
features_2=pd.read_csv(r'C:\Users\Asus\PredictingRatings\data\features2.csv')
features_4=pd.read_csv(r'C:\Users\Asus\PredictingRatings\data\features4.csv')

In [4]:
X_train_mean_1, X_test_mean_1, Y_train_mean_1, Y_test_mean_1 =ms.train_test_split(features_1, elos_1['MeanElos'], test_size = 0.3,random_state=rand_st)
X_train_diff_1, X_test_diff_1, Y_train_diff_1, Y_test_diff_1 =ms.train_test_split(features_1, elos_1['DiffElos'], test_size = 0.3,random_state=rand_st)

X_train_mean_2, X_test_mean_2, Y_train_mean_2, Y_test_mean_2 =ms.train_test_split(features_2, elos_2['MeanElos'], test_size = 0.3,random_state=rand_st)
X_train_diff_2, X_test_diff_2, Y_train_diff_2, Y_test_diff_2 =ms.train_test_split(features_2, elos_2['DiffElos'], test_size = 0.3,random_state=rand_st)

X_train_mean_3, X_test_mean_3, Y_train_mean_3, Y_test_mean_3 =ms.train_test_split(features_3, elos_3['MeanElos'], test_size = 0.3,random_state=rand_st)
X_train_diff_3, X_test_diff_3, Y_train_diff_3, Y_test_diff_3 =ms.train_test_split(features_3, elos_3['DiffElos'], test_size = 0.3,random_state=rand_st)

X_train_mean_4, X_test_mean_4, Y_train_mean_4, Y_test_mean_4 =ms.train_test_split(features_4, elos_4['MeanElos'], test_size = 0.3,random_state=rand_st)
X_train_diff_4, X_test_diff_4, Y_train_diff_4, Y_test_diff_4 =ms.train_test_split(features_4, elos_4['DiffElos'], test_size = 0.3,random_state=rand_st)

In [5]:
model_diff=VotingRegressor(estimators=[
                        ('rf_vt',RandomForestRegressor(bootstrap=True, criterion='mae', max_depth=None,
                      max_features=1, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=5, min_samples_split=25,
                      min_weight_fraction_leaf=0.0, n_estimators=500, n_jobs=-1,
                      oob_score=True, random_state=4, verbose=0,
                      warm_start=False)),
    
                             ('svr_vt', SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='auto',
                                    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)),
                             ('lr_vt',LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False))],
                           
                n_jobs=-1,weights=None)

In [6]:
model_mean=VotingRegressor(estimators=[
                        ('rf_vt',RandomForestRegressor(bootstrap=False, criterion='mae', max_depth=None,
                      max_features=2, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=5,
                      min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=-1,
                      oob_score=False, random_state=4, verbose=0,
                      warm_start=False)),
    
                             ('svr_vt', SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='auto',
                                    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)),
                             ('lr_vt',LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False))],
                           
                 n_jobs=-1,weights=None)

In [7]:
def find_mean_error(pred,elos):
    er=0
    count=0
    for ind, val in enumerate(pred):
        er+=abs(elos.iloc[ind,0]-val[0])
        er+=abs(elos.iloc[ind,1]-val[1])
        count+=2
    return round(er/count,3)

def find_abs_error(pred,elos):
    er=0
    for ind, val in enumerate(pred):
        er+=abs(elos.iloc[ind,0]-val[0])
        er+=abs(elos.iloc[ind,1]-val[1])
    return round(er,3)

def find_root_mean_squared_error(pred,elos):
    er=0
    count=0
    for ind, val in enumerate(pred):
        er+=(elos.iloc[ind,0]-val[0])**2
        er+=(elos.iloc[ind,1]-val[1])**2
        count+=2
    return round((er/count)**0.5,3)

In [8]:
def predictions_report(pred_target,predictions):
    mae_report=mae(pred_target,predictions)
    print ('MAE = {:.3f}'.format(mae_report))
    
    rmse_report = (mse(pred_target, predictions))**0.5
    print ('RMSE = {:.3f}'.format(rmse_report))
    
    corr_coef = pearsonr(pred_target,predictions)
    print("Correlation coefficient = {:.3f}".format(corr_coef[0]))
    
    return (mae_report,rmse_report,corr_coef)

### Подбор модели для 1 кластера

In [9]:
start1 = time.time()

voting1=model_mean.fit(X_train_mean_1,Y_train_mean_1)
voting_pred1=voting1.predict(X_test_mean_1)
scores1=predictions_report(Y_test_mean_1,voting_pred1)
full_time1=round(time.time() - start1,3)
print(full_time1, 'c')

MAE = 176.556
RMSE = 222.677
Correlation coefficient = 0.437
305.072 c


In [10]:
output = output.append(pd.Series([str(model_mean),
                                 scores1[0],scores1[1], 
                                 scores1[2][0],full_time1,'Обучение на среднем ЭЛО по 1 кластеру'],
                                 index=output.columns ),
                      ignore_index=True)


output.to_excel(r'C:\Users\Asus\PredictingRatings\models\class_results.xlsx',index=False)
output

,Модель,MAE,RMSE,Коэффициент корр. Пирсона,Время работы (c),Комментарий
0,"LinearRegression(copy_X=True, fit_intercept=Tr...",172.744949,217.273742,0.428199,1.174,Обучение на среднем ЭЛО по кластеру 1
1,"LinearRegression(copy_X=True, fit_intercept=Tr...",150.440190,192.130211,0.572216,0.113,Обучение на разнице ЭЛО по начинающим
2,"LinearRegression(copy_X=True, fit_intercept=Tr...",166.629660,207.827945,0.569104,0.107,Обучение на среднем ЭЛО по 1 кластеру
3,"LinearRegression(copy_X=True, fit_intercept=Tr...",146.568185,187.970319,0.641182,0.008,Обучение на разнице в ЭЛО по 2 кластеру
4,"LinearRegression(copy_X=True, fit_intercept=Tr...",117.564897,150.552498,0.831412,0.009,Обучение на среднем ЭЛО по 3 кластеру
5,"LinearRegression(copy_X=True, fit_intercept=Tr...",95.694204,120.219588,0.873861,0.008,Обучение на разнице в ЭЛО по 3 кластеру
6,"LinearRegression(copy_X=True, fit_intercept=Tr...",172.852117,217.358897,0.509381,0.013,Обучение на среднем ЭЛО по 4 кластеру
7,"LinearRegression(copy_X=True, fit_intercept=Tr...",149.357283,190.437205,0.594318,0.012,Обучение на разнице в ЭЛО по 4 кластеру
8,"AdaBoostRegressor(base_estimator=None, learnin...",179.646548,226.247606,0.377671,92.545,Обучение на среднем ЭЛО по 1 кластеру
9,"AdaBoostRegressor(base_estimator=None, learnin...",156.774713,200.334686,0.535215,10.463,Обучение на разнице ЭЛО по 1 кластеру


In [11]:
start2 = time.time()

voting2=model_diff.fit(X_train_diff_1,Y_train_diff_1)
voting_pred2=voting2.predict(X_test_mean_1)
scores2=predictions_report(Y_test_diff_1,voting_pred2)
full_time2=round(time.time() - start2,3)
print(full_time2, 'c')

MAE = 159.374
RMSE = 199.365
Correlation coefficient = 0.569
215.346 c


In [12]:
output = output.append(pd.Series([str(model_diff),
                                 scores2[0],scores2[1], 
                                 scores2[2][0],full_time2,'Обучение на разнице ЭЛО по 1 кластеру'],
                                 index=output.columns ),
                      ignore_index=True)


output

,Модель,MAE,RMSE,Коэффициент корр. Пирсона,Время работы (c),Комментарий
0,"LinearRegression(copy_X=True, fit_intercept=Tr...",172.744949,217.273742,0.428199,1.174,Обучение на среднем ЭЛО по кластеру 1
1,"LinearRegression(copy_X=True, fit_intercept=Tr...",150.440190,192.130211,0.572216,0.113,Обучение на разнице ЭЛО по начинающим
2,"LinearRegression(copy_X=True, fit_intercept=Tr...",166.629660,207.827945,0.569104,0.107,Обучение на среднем ЭЛО по 1 кластеру
3,"LinearRegression(copy_X=True, fit_intercept=Tr...",146.568185,187.970319,0.641182,0.008,Обучение на разнице в ЭЛО по 2 кластеру
4,"LinearRegression(copy_X=True, fit_intercept=Tr...",117.564897,150.552498,0.831412,0.009,Обучение на среднем ЭЛО по 3 кластеру
5,"LinearRegression(copy_X=True, fit_intercept=Tr...",95.694204,120.219588,0.873861,0.008,Обучение на разнице в ЭЛО по 3 кластеру
6,"LinearRegression(copy_X=True, fit_intercept=Tr...",172.852117,217.358897,0.509381,0.013,Обучение на среднем ЭЛО по 4 кластеру
7,"LinearRegression(copy_X=True, fit_intercept=Tr...",149.357283,190.437205,0.594318,0.012,Обучение на разнице в ЭЛО по 4 кластеру
8,"AdaBoostRegressor(base_estimator=None, learnin...",179.646548,226.247606,0.377671,92.545,Обучение на среднем ЭЛО по 1 кластеру
9,"AdaBoostRegressor(base_estimator=None, learnin...",156.774713,200.334686,0.535215,10.463,Обучение на разнице ЭЛО по 1 кластеру


In [13]:
output.to_excel(r'C:\Users\Asus\PredictingRatings\models\class_results.xlsx',index=False)

In [14]:
white_elos_1 = voting_pred1 + voting_pred2/2
black_elos_1 = voting_pred1-voting_pred2/2
pred_elos1=list(zip(white_elos_1,black_elos_1))
print(len(pred_elos1))
pred_elos1[:5]

6577


[(2301.789102522097, 2315.037826376024),
 (2307.8161886237845, 2302.0150471898764),
 (2113.2818605613147, 2208.046934396475),
 (2214.080440596727, 2297.258398397113),
 (2291.98014616674, 2314.073239818645)]

In [15]:
find_abs_error(pred_elos1,elos_1),find_mean_error(pred_elos1,elos_1),find_root_mean_squared_error(pred_elos1,elos_1)

(2906679.216, 220.973, 276.012)

In [16]:
inner_mae_1=find_mean_error(pred_elos1,elos_1)

In [17]:
errors =errors.append(pd.Series(['1 кластер','Voting Regressor',
                  find_abs_error(pred_elos1,elos_1),
                  find_mean_error(pred_elos1,elos_1),
                   find_root_mean_squared_error(pred_elos1,elos_1) ],
                                   index=errors.columns ),
                      ignore_index=True)

errors.to_excel(r'C:\Users\Asus\PredictingRatings\models\class_errors.xlsx',index=False)
errors

,Группа,Модель,Абсолютная ошибка,MAE,RMSE
0,1 Кластер,LinearRegression,8.306192e+06,189.45700,237.56700
1,2 кластер,LinearRegression,2.481410e+05,180.86100,228.08700
2,3 кластер,LinearRegression,5.772583e+04,126.03900,162.09500
3,4 кластер,LinearRegression,5.350485e+05,188.53000,237.30000
4,Общая ошибка,LinearRegression,2.286777e+06,171.22175,216.26225
5,1 кластер,AdaBoost,3.217283e+06,244.58600,306.16900
6,2 кластер,AdaBoost,2.810322e+05,682.11700,886.27600
7,3 кластер,AdaBoost,5.346101e+04,387.39900,479.34900
8,4 кластер,AdaBoost,3.108843e+05,364.88800,454.87300
9,Общая ошибка,AdaBoost,9.656651e+05,419.74750,531.66675


In [18]:
errors.to_excel(r'C:\Users\Asus\PredictingRatings\models\class_errors.xlsx',index=False)

### Подбор модели для 2 кластера

In [19]:
start3 = time.time()

voting3=model_mean.fit(X_train_mean_2,Y_train_mean_2)
voting_pred3=voting3.predict(X_test_mean_2)
scores3=predictions_report(Y_test_mean_2,voting_pred3)
full_time3=round(time.time() - start3,3)
print(full_time3, 'c')

MAE = 177.183
RMSE = 231.862
Correlation coefficient = 0.339
2.855 c


In [20]:
output = output.append(pd.Series([str(model_mean),
                                 scores3[0],scores3[1], 
                                 scores3[2][0],full_time3,'Обучение на среднем ЭЛО по 2 кластеру'],
                                 index=output.columns ),
                      ignore_index=True)


output.to_excel(r'C:\Users\Asus\PredictingRatings\models\class_results.xlsx',index=False)
output

,Модель,MAE,RMSE,Коэффициент корр. Пирсона,Время работы (c),Комментарий
0,"LinearRegression(copy_X=True, fit_intercept=Tr...",172.744949,217.273742,0.428199,1.174,Обучение на среднем ЭЛО по кластеру 1
1,"LinearRegression(copy_X=True, fit_intercept=Tr...",150.440190,192.130211,0.572216,0.113,Обучение на разнице ЭЛО по начинающим
2,"LinearRegression(copy_X=True, fit_intercept=Tr...",166.629660,207.827945,0.569104,0.107,Обучение на среднем ЭЛО по 1 кластеру
3,"LinearRegression(copy_X=True, fit_intercept=Tr...",146.568185,187.970319,0.641182,0.008,Обучение на разнице в ЭЛО по 2 кластеру
4,"LinearRegression(copy_X=True, fit_intercept=Tr...",117.564897,150.552498,0.831412,0.009,Обучение на среднем ЭЛО по 3 кластеру
5,"LinearRegression(copy_X=True, fit_intercept=Tr...",95.694204,120.219588,0.873861,0.008,Обучение на разнице в ЭЛО по 3 кластеру
6,"LinearRegression(copy_X=True, fit_intercept=Tr...",172.852117,217.358897,0.509381,0.013,Обучение на среднем ЭЛО по 4 кластеру
7,"LinearRegression(copy_X=True, fit_intercept=Tr...",149.357283,190.437205,0.594318,0.012,Обучение на разнице в ЭЛО по 4 кластеру
8,"AdaBoostRegressor(base_estimator=None, learnin...",179.646548,226.247606,0.377671,92.545,Обучение на среднем ЭЛО по 1 кластеру
9,"AdaBoostRegressor(base_estimator=None, learnin...",156.774713,200.334686,0.535215,10.463,Обучение на разнице ЭЛО по 1 кластеру


In [21]:
start4 = time.time()

voting4=model_diff.fit(X_train_diff_2,Y_train_diff_2)
voting_pred4=voting4.predict(X_test_mean_2)
scores4=predictions_report(Y_test_diff_2,voting_pred4)
full_time4=round(time.time() - start4,3)
print(full_time4, 'c')

MAE = 183.575
RMSE = 224.409
Correlation coefficient = 0.380
1.227 c


In [22]:
output = output.append(pd.Series([str(model_diff),
                                 scores4[0],scores4[1], 
                                 scores4[2][0],full_time4,'Обучение на разнице ЭЛО по 2 кластеру'],
                                 index=output.columns ),
                      ignore_index=True)


output

,Модель,MAE,RMSE,Коэффициент корр. Пирсона,Время работы (c),Комментарий
0,"LinearRegression(copy_X=True, fit_intercept=Tr...",172.744949,217.273742,0.428199,1.174,Обучение на среднем ЭЛО по кластеру 1
1,"LinearRegression(copy_X=True, fit_intercept=Tr...",150.440190,192.130211,0.572216,0.113,Обучение на разнице ЭЛО по начинающим
2,"LinearRegression(copy_X=True, fit_intercept=Tr...",166.629660,207.827945,0.569104,0.107,Обучение на среднем ЭЛО по 1 кластеру
3,"LinearRegression(copy_X=True, fit_intercept=Tr...",146.568185,187.970319,0.641182,0.008,Обучение на разнице в ЭЛО по 2 кластеру
4,"LinearRegression(copy_X=True, fit_intercept=Tr...",117.564897,150.552498,0.831412,0.009,Обучение на среднем ЭЛО по 3 кластеру
5,"LinearRegression(copy_X=True, fit_intercept=Tr...",95.694204,120.219588,0.873861,0.008,Обучение на разнице в ЭЛО по 3 кластеру
6,"LinearRegression(copy_X=True, fit_intercept=Tr...",172.852117,217.358897,0.509381,0.013,Обучение на среднем ЭЛО по 4 кластеру
7,"LinearRegression(copy_X=True, fit_intercept=Tr...",149.357283,190.437205,0.594318,0.012,Обучение на разнице в ЭЛО по 4 кластеру
8,"AdaBoostRegressor(base_estimator=None, learnin...",179.646548,226.247606,0.377671,92.545,Обучение на среднем ЭЛО по 1 кластеру
9,"AdaBoostRegressor(base_estimator=None, learnin...",156.774713,200.334686,0.535215,10.463,Обучение на разнице ЭЛО по 1 кластеру


In [23]:
output.to_excel(r'C:\Users\Asus\PredictingRatings\models\class_results.xlsx',index=False)

In [24]:
white_elos_2 = voting_pred3 + voting_pred3/2
black_elos_2 = voting_pred3-voting_pred4/2
pred_elos2=list(zip(white_elos_2,black_elos_2))
print(len(pred_elos2))
pred_elos2[:5]

206


[(3170.8706861481305, 2144.218853467478),
 (3361.139241298073, 2225.129175310792),
 (3296.852311559286, 2163.7470261428743),
 (3139.6908810901687, 2051.624353033762),
 (3182.168803634204, 2116.431380056422)]

In [25]:
find_abs_error(pred_elos2,elos_2),find_mean_error(pred_elos2,elos_2),find_root_mean_squared_error(pred_elos2,elos_2)

(269732.003, 654.689, 815.023)

In [26]:
inner_mae_2=find_mean_error(pred_elos2,elos_2)

In [27]:
errors =errors.append(pd.Series(['2 кластер','Voting Regressor',
                        find_abs_error(pred_elos2,elos_2),
                        find_mean_error(pred_elos2,elos_2),
                        find_root_mean_squared_error(pred_elos2,elos_2)],
                                index=errors.columns ),
                      ignore_index=True)

errors.to_excel(r'C:\Users\Asus\PredictingRatings\models\class_errors.xlsx',index=False)
errors

,Группа,Модель,Абсолютная ошибка,MAE,RMSE
0,1 Кластер,LinearRegression,8.306192e+06,189.45700,237.56700
1,2 кластер,LinearRegression,2.481410e+05,180.86100,228.08700
2,3 кластер,LinearRegression,5.772583e+04,126.03900,162.09500
3,4 кластер,LinearRegression,5.350485e+05,188.53000,237.30000
4,Общая ошибка,LinearRegression,2.286777e+06,171.22175,216.26225
5,1 кластер,AdaBoost,3.217283e+06,244.58600,306.16900
6,2 кластер,AdaBoost,2.810322e+05,682.11700,886.27600
7,3 кластер,AdaBoost,5.346101e+04,387.39900,479.34900
8,4 кластер,AdaBoost,3.108843e+05,364.88800,454.87300
9,Общая ошибка,AdaBoost,9.656651e+05,419.74750,531.66675


### Подбор модели для 3 кластера

In [28]:
start5 = time.time()

voting5=model_mean.fit(X_train_mean_3,Y_train_mean_3)
voting_pred5=voting5.predict(X_test_mean_3)
scores5=predictions_report(Y_test_mean_3,voting_pred5)
full_time5=round(time.time() - start5,3)
print(full_time5, 'c')

MAE = 2723.069
RMSE = 20628.848
Correlation coefficient = 0.081
0.758 c


In [29]:
output = output.append(pd.Series([str(model_mean),
                                 scores5[0],scores5[1], 
                                 scores5[2][0],full_time5,'Обучение на среднем ЭЛО по 3 кластеру'],
                                 index=output.columns ),
                      ignore_index=True)


output.to_excel(r'C:\Users\Asus\PredictingRatings\models\class_results.xlsx',index=False)
output

,Модель,MAE,RMSE,Коэффициент корр. Пирсона,Время работы (c),Комментарий
0,"LinearRegression(copy_X=True, fit_intercept=Tr...",172.744949,217.273742,0.428199,1.174,Обучение на среднем ЭЛО по кластеру 1
1,"LinearRegression(copy_X=True, fit_intercept=Tr...",150.440190,192.130211,0.572216,0.113,Обучение на разнице ЭЛО по начинающим
2,"LinearRegression(copy_X=True, fit_intercept=Tr...",166.629660,207.827945,0.569104,0.107,Обучение на среднем ЭЛО по 1 кластеру
3,"LinearRegression(copy_X=True, fit_intercept=Tr...",146.568185,187.970319,0.641182,0.008,Обучение на разнице в ЭЛО по 2 кластеру
4,"LinearRegression(copy_X=True, fit_intercept=Tr...",117.564897,150.552498,0.831412,0.009,Обучение на среднем ЭЛО по 3 кластеру
5,"LinearRegression(copy_X=True, fit_intercept=Tr...",95.694204,120.219588,0.873861,0.008,Обучение на разнице в ЭЛО по 3 кластеру
6,"LinearRegression(copy_X=True, fit_intercept=Tr...",172.852117,217.358897,0.509381,0.013,Обучение на среднем ЭЛО по 4 кластеру
7,"LinearRegression(copy_X=True, fit_intercept=Tr...",149.357283,190.437205,0.594318,0.012,Обучение на разнице в ЭЛО по 4 кластеру
8,"AdaBoostRegressor(base_estimator=None, learnin...",179.646548,226.247606,0.377671,92.545,Обучение на среднем ЭЛО по 1 кластеру
9,"AdaBoostRegressor(base_estimator=None, learnin...",156.774713,200.334686,0.535215,10.463,Обучение на разнице ЭЛО по 1 кластеру


In [30]:
start6 = time.time()

voting6=model_diff.fit(X_train_diff_3,Y_train_diff_3)
voting_pred6=voting6.predict(X_test_mean_3)
scores6=predictions_report(Y_test_diff_3,voting_pred6)
full_time6=round(time.time() - start6,3)
print(full_time6, 'c')

MAE = 2014.138
RMSE = 15082.386
Correlation coefficient = -0.153
1.267 c


In [31]:
output = output.append(pd.Series([str(model_diff),
                                 scores6[0],scores6[1], 
                                 scores6[2][0],full_time6,'Обучение на разнице ЭЛО по 3 кластеру'],
                                 index=output.columns ),
                      ignore_index=True)


output

,Модель,MAE,RMSE,Коэффициент корр. Пирсона,Время работы (c),Комментарий
0,"LinearRegression(copy_X=True, fit_intercept=Tr...",172.744949,217.273742,0.428199,1.174,Обучение на среднем ЭЛО по кластеру 1
1,"LinearRegression(copy_X=True, fit_intercept=Tr...",150.440190,192.130211,0.572216,0.113,Обучение на разнице ЭЛО по начинающим
2,"LinearRegression(copy_X=True, fit_intercept=Tr...",166.629660,207.827945,0.569104,0.107,Обучение на среднем ЭЛО по 1 кластеру
3,"LinearRegression(copy_X=True, fit_intercept=Tr...",146.568185,187.970319,0.641182,0.008,Обучение на разнице в ЭЛО по 2 кластеру
4,"LinearRegression(copy_X=True, fit_intercept=Tr...",117.564897,150.552498,0.831412,0.009,Обучение на среднем ЭЛО по 3 кластеру
5,"LinearRegression(copy_X=True, fit_intercept=Tr...",95.694204,120.219588,0.873861,0.008,Обучение на разнице в ЭЛО по 3 кластеру
6,"LinearRegression(copy_X=True, fit_intercept=Tr...",172.852117,217.358897,0.509381,0.013,Обучение на среднем ЭЛО по 4 кластеру
7,"LinearRegression(copy_X=True, fit_intercept=Tr...",149.357283,190.437205,0.594318,0.012,Обучение на разнице в ЭЛО по 4 кластеру
8,"AdaBoostRegressor(base_estimator=None, learnin...",179.646548,226.247606,0.377671,92.545,Обучение на среднем ЭЛО по 1 кластеру
9,"AdaBoostRegressor(base_estimator=None, learnin...",156.774713,200.334686,0.535215,10.463,Обучение на разнице ЭЛО по 1 кластеру


In [32]:
output.to_excel(r'C:\Users\Asus\PredictingRatings\models\class_results.xlsx',index=False)

In [33]:
white_elos_3 = voting_pred5 + voting_pred6/2
black_elos_3 = voting_pred5-voting_pred6/2
pred_elos3=list(zip(white_elos_3,black_elos_3))
print(len(pred_elos3))
pred_elos3[:5]

69


[(2148.0820081464394, 2017.0984094347248),
 (2142.638491842003, 2170.9427594794784),
 (1698.6961472759444, 2217.1271867216174),
 (2164.6644973568855, 2071.771005950749),
 (2231.6254739744995, 2326.285482700237)]

In [34]:
find_abs_error(pred_elos3,elos_3),find_mean_error(pred_elos3,elos_3),find_root_mean_squared_error(pred_elos3,elos_3)

(378175.906, 2740.405, 21909.396)

In [35]:
inner_mae_3=find_mean_error(pred_elos3,elos_3)

In [36]:
errors =errors.append(pd.Series(['3 кластер','Voting Regressor',
                                 find_abs_error(pred_elos3,elos_3),
                                 find_mean_error(pred_elos3,elos_3),
                                 find_root_mean_squared_error(pred_elos3,elos_3)],
                                index=errors.columns ),
                      ignore_index=True)

errors.to_excel(r'C:\Users\Asus\PredictingRatings\models\class_errors.xlsx',index=False)
errors

,Группа,Модель,Абсолютная ошибка,MAE,RMSE
0,1 Кластер,LinearRegression,8.306192e+06,189.45700,237.56700
1,2 кластер,LinearRegression,2.481410e+05,180.86100,228.08700
2,3 кластер,LinearRegression,5.772583e+04,126.03900,162.09500
3,4 кластер,LinearRegression,5.350485e+05,188.53000,237.30000
4,Общая ошибка,LinearRegression,2.286777e+06,171.22175,216.26225
5,1 кластер,AdaBoost,3.217283e+06,244.58600,306.16900
6,2 кластер,AdaBoost,2.810322e+05,682.11700,886.27600
7,3 кластер,AdaBoost,5.346101e+04,387.39900,479.34900
8,4 кластер,AdaBoost,3.108843e+05,364.88800,454.87300
9,Общая ошибка,AdaBoost,9.656651e+05,419.74750,531.66675


### Подбор модели для 4 кластера

In [37]:
start7 = time.time()

voting7=model_mean.fit(X_train_mean_4,Y_train_mean_4)
voting_pred7=voting7.predict(X_test_mean_4)
scores7=predictions_report(Y_test_mean_4,voting_pred7)
full_time7=round(time.time() - start7,3)
print(full_time7, 'c')

MAE = 184.330
RMSE = 231.242
Correlation coefficient = 0.391
3.11 c


In [38]:
output = output.append(pd.Series([str(model_mean),
                                 scores7[0],scores7[1], 
                                 scores7[2][0],full_time7,'Обучение на среднем ЭЛО по 4 кластеру'],
                                 index=output.columns ),
                      ignore_index=True)


output.to_excel(r'C:\Users\Asus\PredictingRatings\models\class_results.xlsx',index=False)
output

,Модель,MAE,RMSE,Коэффициент корр. Пирсона,Время работы (c),Комментарий
0,"LinearRegression(copy_X=True, fit_intercept=Tr...",172.744949,217.273742,0.428199,1.174,Обучение на среднем ЭЛО по кластеру 1
1,"LinearRegression(copy_X=True, fit_intercept=Tr...",150.440190,192.130211,0.572216,0.113,Обучение на разнице ЭЛО по начинающим
2,"LinearRegression(copy_X=True, fit_intercept=Tr...",166.629660,207.827945,0.569104,0.107,Обучение на среднем ЭЛО по 1 кластеру
3,"LinearRegression(copy_X=True, fit_intercept=Tr...",146.568185,187.970319,0.641182,0.008,Обучение на разнице в ЭЛО по 2 кластеру
4,"LinearRegression(copy_X=True, fit_intercept=Tr...",117.564897,150.552498,0.831412,0.009,Обучение на среднем ЭЛО по 3 кластеру
5,"LinearRegression(copy_X=True, fit_intercept=Tr...",95.694204,120.219588,0.873861,0.008,Обучение на разнице в ЭЛО по 3 кластеру
6,"LinearRegression(copy_X=True, fit_intercept=Tr...",172.852117,217.358897,0.509381,0.013,Обучение на среднем ЭЛО по 4 кластеру
7,"LinearRegression(copy_X=True, fit_intercept=Tr...",149.357283,190.437205,0.594318,0.012,Обучение на разнице в ЭЛО по 4 кластеру
8,"AdaBoostRegressor(base_estimator=None, learnin...",179.646548,226.247606,0.377671,92.545,Обучение на среднем ЭЛО по 1 кластеру
9,"AdaBoostRegressor(base_estimator=None, learnin...",156.774713,200.334686,0.535215,10.463,Обучение на разнице ЭЛО по 1 кластеру


In [39]:
start8 = time.time()

voting8=model_diff.fit(X_train_diff_4,Y_train_diff_4)
voting_pred8=voting8.predict(X_test_mean_4)
scores8=predictions_report(Y_test_diff_4,voting_pred8)
full_time8=round(time.time() - start8,3)
print(full_time8, 'c')

MAE = 176.091
RMSE = 308.776
Correlation coefficient = 0.109
2.079 c


In [40]:
output = output.append(pd.Series([str(model_diff),
                                 scores8[0],scores8[1], 
                                 scores8[2][0],full_time8,'Обучение на разнице ЭЛО по 4 кластеру'],
                                 index=output.columns ),
                      ignore_index=True)


output.to_excel(r'C:\Users\Asus\PredictingRatings\models\class_results.xlsx',index=False)
output

,Модель,MAE,RMSE,Коэффициент корр. Пирсона,Время работы (c),Комментарий
0,"LinearRegression(copy_X=True, fit_intercept=Tr...",172.744949,217.273742,0.428199,1.174,Обучение на среднем ЭЛО по кластеру 1
1,"LinearRegression(copy_X=True, fit_intercept=Tr...",150.440190,192.130211,0.572216,0.113,Обучение на разнице ЭЛО по начинающим
2,"LinearRegression(copy_X=True, fit_intercept=Tr...",166.629660,207.827945,0.569104,0.107,Обучение на среднем ЭЛО по 1 кластеру
3,"LinearRegression(copy_X=True, fit_intercept=Tr...",146.568185,187.970319,0.641182,0.008,Обучение на разнице в ЭЛО по 2 кластеру
4,"LinearRegression(copy_X=True, fit_intercept=Tr...",117.564897,150.552498,0.831412,0.009,Обучение на среднем ЭЛО по 3 кластеру
5,"LinearRegression(copy_X=True, fit_intercept=Tr...",95.694204,120.219588,0.873861,0.008,Обучение на разнице в ЭЛО по 3 кластеру
6,"LinearRegression(copy_X=True, fit_intercept=Tr...",172.852117,217.358897,0.509381,0.013,Обучение на среднем ЭЛО по 4 кластеру
7,"LinearRegression(copy_X=True, fit_intercept=Tr...",149.357283,190.437205,0.594318,0.012,Обучение на разнице в ЭЛО по 4 кластеру
8,"AdaBoostRegressor(base_estimator=None, learnin...",179.646548,226.247606,0.377671,92.545,Обучение на среднем ЭЛО по 1 кластеру
9,"AdaBoostRegressor(base_estimator=None, learnin...",156.774713,200.334686,0.535215,10.463,Обучение на разнице ЭЛО по 1 кластеру


In [41]:
white_elos_4 = voting_pred7 + voting_pred8/2
black_elos_4 = voting_pred7-voting_pred8/2
pred_elos4=list(zip(white_elos_4,black_elos_4))
print(len(pred_elos4))
pred_elos4[:5]

426


[(2135.3791913464383, 2162.9301570862917),
 (2198.3750921567594, 2238.4684814959023),
 (2255.5491906849134, 2153.367350621507),
 (2176.2817884358446, 2176.234319067097),
 (2214.608997817235, 2152.9820404043066)]

In [42]:
find_abs_error(pred_elos4,elos_4),find_mean_error(pred_elos4,elos_4),find_root_mean_squared_error(pred_elos4,elos_4)

(216562.853, 254.182, 324.89)

In [43]:
inner_mae_4=find_mean_error(pred_elos4,elos_4)

In [44]:
errors =errors.append(pd.Series(['4 кластер','Voting Regressor',find_abs_error(pred_elos4,elos_4),
                                 find_mean_error(pred_elos4,elos_4),
                                 find_root_mean_squared_error(pred_elos4,elos_4)],
                                index=errors.columns ),
                      ignore_index=True)

errors.to_excel(r'C:\Users\Asus\PredictingRatings\models\class_errors.xlsx',index=False)
errors

,Группа,Модель,Абсолютная ошибка,MAE,RMSE
0,1 Кластер,LinearRegression,8.306192e+06,189.45700,237.56700
1,2 кластер,LinearRegression,2.481410e+05,180.86100,228.08700
2,3 кластер,LinearRegression,5.772583e+04,126.03900,162.09500
3,4 кластер,LinearRegression,5.350485e+05,188.53000,237.30000
4,Общая ошибка,LinearRegression,2.286777e+06,171.22175,216.26225
5,1 кластер,AdaBoost,3.217283e+06,244.58600,306.16900
6,2 кластер,AdaBoost,2.810322e+05,682.11700,886.27600
7,3 кластер,AdaBoost,5.346101e+04,387.39900,479.34900
8,4 кластер,AdaBoost,3.108843e+05,364.88800,454.87300
9,Общая ошибка,AdaBoost,9.656651e+05,419.74750,531.66675


In [45]:
voting_mae=(inner_mae_4+inner_mae_3+inner_mae_2+inner_mae_1)/4
voting_mae

967.56225

In [46]:
voting_abs=(find_abs_error(pred_elos1,elos_1)+
        find_abs_error(pred_elos2,elos_2)+
        find_abs_error(pred_elos3,elos_3)+
        find_abs_error(pred_elos4,elos_4))/4
voting_abs

942787.4945

In [47]:
voting_rmse=(find_root_mean_squared_error(pred_elos1,elos_1)+
         find_root_mean_squared_error(pred_elos2,elos_2)+
         find_root_mean_squared_error(pred_elos3,elos_3)+
         find_root_mean_squared_error(pred_elos4,elos_4))/4
voting_rmse

5831.33025

In [48]:
errors = errors.append(pd.Series(['Общая ошибка','Voting Regressor',
                  voting_abs,
                  voting_mae,
                 voting_rmse],index=errors.columns) ,ignore_index=True)

errors.to_excel(r'C:\Users\Asus\PredictingRatings\models\class_errors.xlsx',index=False)
errors

,Группа,Модель,Абсолютная ошибка,MAE,RMSE
0,1 Кластер,LinearRegression,8.306192e+06,189.45700,237.56700
1,2 кластер,LinearRegression,2.481410e+05,180.86100,228.08700
2,3 кластер,LinearRegression,5.772583e+04,126.03900,162.09500
3,4 кластер,LinearRegression,5.350485e+05,188.53000,237.30000
4,Общая ошибка,LinearRegression,2.286777e+06,171.22175,216.26225
5,1 кластер,AdaBoost,3.217283e+06,244.58600,306.16900
6,2 кластер,AdaBoost,2.810322e+05,682.11700,886.27600
7,3 кластер,AdaBoost,5.346101e+04,387.39900,479.34900
8,4 кластер,AdaBoost,3.108843e+05,364.88800,454.87300
9,Общая ошибка,AdaBoost,9.656651e+05,419.74750,531.66675
